# Procesamiento del texto, extraemos todo el texto del pdf y lo guardamos en un archivo de texto.

In [ ]:
import fitz  
import os

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    
    for page in doc:
        text += page.get_text("text") + "\n"  # Extrae texto de cada página
    
    return text.strip()

# Ruta de tu PDF
pdf_path = "../data/raw/leyes_ajedrez.pdf"  

# Extraer texto
texto_extraido = extract_text_from_pdf(pdf_path)

# Crear carpeta si no existe    
output_dir = "../data/cooked"
os.makedirs(output_dir, exist_ok=True)

# Guardar el texto en un archivo dentro de 'data/cooked'
output_path = os.path.join(output_dir, "texto_extraido.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(texto_extraido)

print(f"✅ Extracción completada. El texto se guardó en '{output_path}'.")

✅ Extracción completada. El texto se guardó en '../data/cooked\texto_extraido.txt'.


# La base de datos elegida, ha sido Faiss

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar el modelo preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')  # Puedes cambiar el modelo si lo deseas

# Leer el archivo de texto
with open('../data/cooked/texto_extraido.txt', 'r') as f:
    lines = f.readlines()

# Generar embeddings para cada línea del archivo
embeddings = model.encode(lines, convert_to_numpy=True)

# Ver el primer embedding para verificar
print("Embedding de la primera línea:", embeddings[0])

In [4]:
import faiss

# Dimensión de los embeddings (depende del modelo que uses, por ejemplo, 384 para 'all-MiniLM-L6-v2')
dim = embeddings.shape[1]

# Crear el índice FAISS (usamos IndexFlatL2 para distancia euclidiana)
index = faiss.IndexFlatL2(dim)

# Agregar los embeddings al índice
index.add(embeddings)

# Verifica cuántos vectores se han añadido
print("Número de vectores en el índice:", index.ntotal)

Número de vectores en el índice: 1827


In [8]:
consulta = ["cuando se gana una partida"]
consulta_embedding = model.encode(consulta, convert_to_numpy=True)

In [9]:
# Buscar los 5 vectores más cercanos
k = 5
distances, indices = index.search(consulta_embedding, k)

print("Índices de los vectores más cercanos:", indices)
print("Distancias de los vectores más cercanos:", distances)

Índices de los vectores más cercanos: [[483 760 937 655 370]]
Distancias de los vectores más cercanos: [[0.5913608  0.5915155  0.595267   0.61422104 0.6148168 ]]


In [10]:
# Guardar el índice
faiss.write_index(index, "mi_indice.index")

# Cargar el índice desde el archivo
index_cargado = faiss.read_index("mi_indice.index")

In [11]:
import faiss

#Comprobamos la dimensión de los embeddings
dim = embeddings.shape[1]

#Creamos un índice
index = faiss.IndexFlatL2(dim)  

# Agregar los embeddings al índice
index.add(embeddings)  # embeddings es un array numpy de tamaño (número de textos, dimensiones del embedding)
print(f"Se han agregado {embeddings.shape[0]} embeddings al índice.")

Se han agregado 1827 embeddings al índice.
